# Script #4 - GEOTIFF Reprojector

*Mike Huff, 2025*

https://github.com/m-huff

If you don't want to reproject this data into a more "normal" map projection, you can skip this script! The "What GOES Around" map I published on my website, huffmaps.com, keeps the data in its native projection (the GEOS geostationary satellite projection, that just needs to be adjusted by the equitorial positioning of the satellite, which varies between GOES-West and GOES-East imagery).

This script converts your cleaned, interpolated GEOTIFF into a georeferenced GeoTIFF and reprojects it to a global coordinate system for mapping and analysis. It begins by opening an EXAMPLE NetCDF file using xarray (note: use one of the original NetCDF files you've downloaded straight from NASA's EarthData Downloader for this), extracting the Total_Optical_energy variable, and reading the GOES projection metadata (e.g., satellite height, sweep angle, and projection origin). It then constructs a proper geostationary CRS definition and converts the dataset’s x/y coordinates from radians to meters. Using rasterio, it opens an existing GeoTIFF containing GLM data values, assigns the new CRS and affine transform, and saves a properly georeferenced version. Finally, the script reprojects the output raster to the WGS 84 coordinate system (EPSG:4326), making it compatible with most GIS platforms.

How to use:
1. Update the file paths (netcdf_path, raster_path, etc.) to match your local data.
2. Ensure that your NetCDF file contains the goes_imager_projection variable and the variable of interest (e.g., Total_Optical_energy).

The script will produce two GeoTIFFs — one georeferenced in the GOES projection and another reprojected to WGS 84 — ready for visualization or integration into GIS workflows.

In [ ]:
%pip install rasterio xarray

import xarray as xr
import rasterio
from rasterio.transform import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling

### VARIABLES TO CONTROL THE SCRIPT
### THESE ARE DESCRIBED IN THE MARKDOWN CELL ABOVE
netcdf_path = r"E:\GOES-R Lightning Data\EAST-2025\OR_GLM-L3-GLMF-M6_G19_s202518200040000_e202518200050000_c20251820006050.nc"
raster_path = r"E:\GOES-R Lightning Data\EAST-RASTERS\east_mean_energy_2025_cleaned.tif"
output_raster = r"E:\GOES-R Lightning Data\EAST-RASTERS\east_mean_energy_2025_georef.tif"
reproj_raster = r"E:\GOES-R Lightning Data\EAST-RASTERS\east_mean_energy_2025_wgs84.tif"

ds = xr.open_dataset(netcdf_path)
var_name = 'Total_Optical_energy'
da_nc = ds[var_name]

proj_var = ds['goes_imager_projection']
h = proj_var.perspective_point_height 
lon_0 = proj_var.longitude_of_projection_origin
sweep = proj_var.sweep_angle_axis
a = proj_var.semi_major_axis
b = proj_var.semi_minor_axis

crs_string = (
    f"+proj=geos +h={h} +lon_0={lon_0} +sweep={sweep} "
    f"+a={a} +b={b} +units=m +no_defs"
)


x = ds['x'].values  # in radians
y = ds['y'].values  # in radians
x_m = x * h
y_m = y * h
pixel_width = x_m[1] - x_m[0]
pixel_height = y_m[1] - y_m[0]
x_min = x_m[0] - pixel_width / 2
y_max = y_m[0] - pixel_height / 2
transform = Affine(pixel_width, 0, x_min, 0, pixel_height, y_max)

with rasterio.open(raster_path, 'r') as src:
    data = src.read(1)
    profile = src.profile

profile.update({
    'crs': crs_string,
    'transform': transform
})

with rasterio.open(output_raster, 'w', **profile) as dst:
    dst.write(data, 1)

print(f"Saved georeferenced raster to: {output_raster}")

dst_crs = 'EPSG:4326'
with rasterio.open(output_raster) as src:
    transform_new, width_new, height_new = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    profile_new = src.profile.copy()
    profile_new.update({
        'crs': dst_crs,
        'transform': transform_new,
        'width': width_new,
        'height': height_new
    })

    data_new = rasterio.open(reproj_raster, 'w', **profile_new)
    reproject(
        source=rasterio.band(src, 1),
        destination=rasterio.band(data_new, 1),
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=transform_new,
        dst_crs=dst_crs,
        resampling=Resampling.nearest
    )
    data_new.close()

print(f"Saved reprojected raster to: {reproj_raster}")
